In [1]:
import csv
import os
import pickle
import re
import sys
import numpy as np

from joblib import Parallel, delayed

from difflib import SequenceMatcher,get_close_matches

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Input, Flatten, Activation
from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalAvgPool1D, GlobalMaxPooling1D
from keras.models import Model, Sequential

from numpy import linalg as LA

from scipy import spatial

from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from tqdm import *

module_path = os.path.abspath(os.path.join('../helpers/'))
if module_path not in sys.path:
    sys.path.append(module_path)

# my stuff in the helpers/ directory
import embeddings_helper, files_helper, texts_helper, metrics_helper, tags_helper,cobrinha_helper

from cobrinha_helper import evaluate_cobrinha


Using TensorFlow backend.


In [2]:
SEED=np.random.randint(1,1000)
SEED

759

In [3]:
np.random.seed(SEED)

In [4]:
PICKLE_DIR_ROOT = "/media/felipe/SSD_VOLUME/auto-tagger/data/tag-hierarchy/"

In [5]:
texts, labels = files_helper.read_stackoverflow_sample_stanford_tokenized("Medium-Small-Sample-Posts-Shuffled",ssd=True)

In [6]:
MAX_NB_WORDS = 10000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
LABELS_MIN_DOC_COUNT = int(20)
BATCH_SIZE=32
EMBEDDING_DIM=100
NUM_EPOCHS=10
TOKENIZER_FILTERS='' # I will perform tokenization myself

In [7]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS,
                     filters=TOKENIZER_FILTERS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# word => word_index_position
word_index = tokenizer.word_index

# word_index_position => word
inverse_word_index = texts_helper.build_inverse_word_index(word_index)

In [8]:
truncated_labels = tags_helper.truncate_labels(labels,LABELS_MIN_DOC_COUNT)

In [9]:
lb = preprocessing.MultiLabelBinarizer()
binary_labels = lb.fit_transform(truncated_labels)

In [10]:
# make each document (sequence of word indices) be truncated to 
# MAX_SEQUENCE_LENGTH
tokenized_texts = []

for seq in sequences:
    truncated_seq = seq[:MAX_SEQUENCE_LENGTH]
    tokenized_txt = " ".join([inverse_word_index[idx] for idx in truncated_seq])
    tokenized_texts.append(tokenized_txt)

In [11]:
# extracting IDF weights to weight the embeddings

# snooping
vect = TfidfVectorizer(max_features=MAX_NB_WORDS).fit(tokenized_texts)

feature_names = vect.get_feature_names()
idf = vect.idf_

# word => word IDF
idf_index = dict(zip(vect.get_feature_names(), idf))

In [12]:
document_vectors = vect.transform(tokenized_texts)

In [13]:
NUM_WORDS = document_vectors[0].shape[1]

In [14]:
# document_id => [tag1, tag2, tag3, ...]
document_tag_index = tags_helper.get_document_tag_index(binary_labels,lb.classes_)

# tag => [document_id1, document_id2, document_id3, ...]
tag_document_index = tags_helper.get_tag_document_index(binary_labels, lb.classes_)

### build tag vocabulary

In [15]:
tag_vocabulary = lb.classes_

In [16]:
len(tag_vocabulary)

1704

In [17]:
pickle.dump(tag_vocabulary,open(PICKLE_DIR_ROOT+"/tag_vocabulary.p","wb"))

In [18]:
i = 2
tokenized_texts[i],document_tag_index[i]

("use with gulp . i read about gulp and was quite taken by the . i want to try it out for myself but i am running into a little problem . i am used to using with grunt and i have no idea how to get to play nice with gulp . i 've come across this article which suggests there is no need for a plugin when using gulp . unfortunately it does n't really explain how to go about it and the example it links to does n't help me much . is there anyone who knows how to go about this ?",
 ['gulp', 'jekyll'])

### build tag_vectors_index

> if a tag is the ONLY tag assigned to some document, that's probably a more representative document of this tag, than if this tag were only one among many others


In [19]:
# tag => tag_vector
tag_vectors_index = dict()

for (tag,document_ids) in tag_document_index.items():
    
    tag_vector = np.zeros(NUM_WORDS)
    
    for document_id in document_ids:
        document_vector = document_vectors[document_id]
        num_tags = len(document_tag_index[document_id])


        weight = 1.0 / num_tags

        weighted_document_vector = document_vector * weight
        
        tag_vector = tag_vector + weighted_document_vector
        
    tag_vectors_index[tag] = np.asarray(tag_vector).ravel()

In [20]:
pickle.dump(tag_vectors_index,open(PICKLE_DIR_ROOT+"/tag_vectors_index.p","wb"))

In [21]:
normalized_tag_vectors_index = dict()    

## THIS SUCKS.. WHY?
## normalize vectors to make cosine similarity more accurate  
for (tag,tag_vector) in tag_vectors_index.items():    
    normalized_tag_vectors_index[tag] = LA.norm(tag_vector, 2)

In [22]:
def _cosine_similarity(a,b):
    return 1 - spatial.distance.cosine(a, b)

In [23]:
target_tags = ['sql-server-2008','sql','sql-server','mysql','database','postgresql','select','join','oracle','tsql','sqlite']

pairs = [ (a,b) for a in target_tags for b in target_tags if a != b ]

In [24]:
for (tag_a,tag_b) in pairs:
    tag_a_vector = tag_vectors_index[tag_a]
    tag_b_vector = tag_vectors_index[tag_b]
    
    sim = _cosine_similarity(tag_a_vector,tag_b_vector)
    
    print("SIM '{}' '{}' => {}".format(tag_a,tag_b,sim))
    

SIM 'sql-server-2008' 'sql' => 0.9349852894653755
SIM 'sql-server-2008' 'sql-server' => 0.9660928141839111
SIM 'sql-server-2008' 'mysql' => 0.8154538869611813
SIM 'sql-server-2008' 'database' => 0.8156094736024496
SIM 'sql-server-2008' 'postgresql' => 0.7553914379145025
SIM 'sql-server-2008' 'select' => 0.7778991611709338
SIM 'sql-server-2008' 'join' => 0.7190127691231447
SIM 'sql-server-2008' 'oracle' => 0.764526362604965
SIM 'sql-server-2008' 'tsql' => 0.9261999555224482
SIM 'sql-server-2008' 'sqlite' => 0.6563355913804496
SIM 'sql' 'sql-server-2008' => 0.9349852894653755
SIM 'sql' 'sql-server' => 0.9460839963190458
SIM 'sql' 'mysql' => 0.906623737639903
SIM 'sql' 'database' => 0.8520139436413494
SIM 'sql' 'postgresql' => 0.8138921551069163
SIM 'sql' 'select' => 0.8654244931904951
SIM 'sql' 'join' => 0.8199456810146458
SIM 'sql' 'oracle' => 0.8284758313803156
SIM 'sql' 'tsql' => 0.9364463457744994
SIM 'sql' 'sqlite' => 0.7008826723192213
SIM 'sql-server' 'sql-server-2008' => 0.966092

if all other tags are, on average, more similar to A than to B, then A is probably more generic than B

should it be normalized by the difference between A and B?

will fail for tags that are below B?

In [25]:
sql_vector = tag_vectors_index['sql']
avg_diff = list()

for (tag_name,tag_vector) in tag_vectors_index.items():
    avg_diff.append(_cosine_similarity(tag_a_vector,sql_vector))
    
np.mean(np.array(avg_diff))

0.7008826723192213

In [26]:
sql_vector = tag_vectors_index['sql-server']
avg_diff = list()

for (tag_name,tag_vector) in tag_vectors_index.items():
    avg_diff.append(_cosine_similarity(tag_a_vector,sql_vector))
    
np.mean(np.array(avg_diff))

0.66694280469786815

In [27]:
sql_vector = tag_vectors_index['sql-server-2008']
avg_diff = list()

for (tag_name,tag_vector) in tag_vectors_index.items():
    avg_diff.append(_cosine_similarity(tag_a_vector,sql_vector))
    
np.mean(np.array(avg_diff))

0.65633559138044983

In [28]:
def get_sim(tag_a,tag_b):
    
    sim = _cosine_similarity(tag_vectors_index[tag_a],tag_vectors_index[tag_b])
    
    return (tag_a,tag_b,sim)

In [29]:
all_tags = [tag for (tag,_) in tag_vectors_index.items()]

In [30]:
sims = Parallel(n_jobs=-1)(delayed(get_sim)(a,b) for a in all_tags for b in all_tags)

In [31]:
sims[:10]

[('swt', 'swt', 1.0),
 ('swt', 'treeview', 0.22646766922432759),
 ('swt', 'tinymce', 0.17243342661698979),
 ('swt', 'rmi', 0.14675991005559186),
 ('swt', 'puppet', 0.14897517413558226),
 ('swt', 'elisp', 0.20731017723506406),
 ('swt', 'cors', 0.18477021462352949),
 ('swt', 'prototypejs', 0.23289653848088243),
 ('swt', 'dependencies', 0.26083028054247825),
 ('swt', 'tornado', 0.14338674336878943)]

In [32]:
similarity_dict = dict()

for tag_a,tag_b, sim in sims:
    
    if similarity_dict.get(tag_a) is None:
        similarity_dict[tag_a] = [(tag_b,sim)]
    else:
        similarity_dict[tag_a].append((tag_b,sim))

In [33]:
similarity_dict["sql"][:20]

[('swt', 0.29319250653441919),
 ('treeview', 0.35661211397222492),
 ('tinymce', 0.28073046054331141),
 ('rmi', 0.22925921098964364),
 ('puppet', 0.26736057867589014),
 ('elisp', 0.36765672430944696),
 ('cors', 0.34093837910864178),
 ('prototypejs', 0.41060220823212745),
 ('dependencies', 0.457743537133385),
 ('tornado', 0.27526369817278074),
 ('serial-port', 0.38575138691499367),
 ('expandablelistview', 0.24961132999371127),
 ('visual-studio-2013', 0.50889254615177959),
 ('shader', 0.31968099882620304),
 ('ecmascript-6', 0.4004323864779884),
 ('repeater', 0.35901714215114788),
 ('insert', 0.72465146895700683),
 ('visual-studio-2012', 0.51403709838615652),
 ('uiscrollview', 0.32240031074668196),
 ('debugging', 0.58591588580601484)]

In [34]:
sorted(similarity_dict["sql"],key=lambda tpl : tpl[1],reverse=True)[1:20]

[('sql-server', 0.94608399631904583),
 ('tsql', 0.93644634577449937),
 ('sql-server-2008', 0.93498528946537551),
 ('mysql', 0.906623737639903),
 ('select', 0.86542449319049508),
 ('sql-server-2005', 0.85241731531134013),
 ('database', 0.85201394364134941),
 ('sql-server-2008-r2', 0.8431796843868572),
 ('oracle', 0.82847583138031555),
 ('sql-server-2012', 0.82132500838792566),
 ('join', 0.81994568101464582),
 ('postgresql', 0.81389215510691626),
 ('ms-access', 0.80221184346494157),
 ('database-design', 0.78301096529451608),
 ('sql-update', 0.77638822429249021),
 ('ms-access-2010', 0.77491311729250856),
 ('group-by', 0.77283620646348683),
 ('subquery', 0.76446459464712535),
 ('query-optimization', 0.74722904760620112)]

In [35]:
sorted_similarity_dict = dict()

for (tag, similarities_to_other_tags) in similarity_dict.items():
    
    sorted_similarities = sorted(similarities_to_other_tags,key=lambda tpl: tpl[1],reverse=True)
    sorted_without_self = sorted_similarities[1:]
    
    sorted_similarity_dict[tag] = sorted_without_self

In [36]:
sorted_similarity_dict["sql"][:20]

[('sql-server', 0.94608399631904583),
 ('tsql', 0.93644634577449937),
 ('sql-server-2008', 0.93498528946537551),
 ('mysql', 0.906623737639903),
 ('select', 0.86542449319049508),
 ('sql-server-2005', 0.85241731531134013),
 ('database', 0.85201394364134941),
 ('sql-server-2008-r2', 0.8431796843868572),
 ('oracle', 0.82847583138031555),
 ('sql-server-2012', 0.82132500838792566),
 ('join', 0.81994568101464582),
 ('postgresql', 0.81389215510691626),
 ('ms-access', 0.80221184346494157),
 ('database-design', 0.78301096529451608),
 ('sql-update', 0.77638822429249021),
 ('ms-access-2010', 0.77491311729250856),
 ('group-by', 0.77283620646348683),
 ('subquery', 0.76446459464712535),
 ('query-optimization', 0.74722904760620112),
 ('oracle-sqldeveloper', 0.74190584312943719)]

In [37]:
pickle.dump(sorted_similarity_dict,open(PICKLE_DIR_ROOT+"/sorted_similarity_dict",'wb'))

### build global similarity index

In [38]:
global_similarity_index = dict()

for tag in tag_vocabulary:
    
    similarities_with_current_tag = list()
    
    for other_tag, similarity_to_other_tag in sorted_similarity_dict[tag]:
        
        if other_tag != tag:
            similarities_with_current_tag.append(similarity_to_other_tag)

    global_avg_sim_wrt_tag = np.array(similarities_with_current_tag).mean()
    
    global_similarity_index[tag] = global_avg_sim_wrt_tag
            
pickle.dump(global_similarity_index,open(PICKLE_DIR_ROOT+"/global_similarity_index.p",'wb'))

In [39]:
# global_similarity_index

In [40]:
sorted(global_similarity_index.items(),key=lambda tpl: tpl[1],reverse=True)[:20]

[('c#', 0.52795604153827957),
 ('.net', 0.52236106858240805),
 ('web', 0.4969226718081366),
 ('c#-4.0', 0.49237019402918486),
 ('performance', 0.49044914996248945),
 ('design', 0.4879139521178063),
 ('java', 0.48654698269451507),
 ('javascript', 0.48598142589366),
 ('asp.net', 0.48340248344416714),
 ('security', 0.48224656500145713),
 ('php', 0.479727720273666),
 ('user-interface', 0.47964381495071595),
 ('.net-4.0', 0.47751789859829202),
 ('html5', 0.47722926814144856),
 ('optimization', 0.4765343297036051),
 ('windows', 0.4754632299158465),
 ('ios', 0.47506434270964637),
 ('cocoa', 0.47290753272968278),
 ('cocoa-touch', 0.47098829707762513),
 ('winforms', 0.4693216276848044)]

In [41]:
evaluate_cobrinha('android','android-service',tag_vectors_index,sorted_similarity_dict, global_similarity_index) 

(0.1386092775528045, 0.68468044509989034)

In [42]:
evaluate_cobrinha('asp.net','asp.net-mvc',tag_vectors_index,sorted_similarity_dict, global_similarity_index) 

(0.034236584287048843, 0.84004162240234292)

In [43]:
evaluate_cobrinha('asp.net-mvc','asp.net-mvc-5',tag_vectors_index,sorted_similarity_dict, global_similarity_index) 

(0.011887487520178763, 0.89472965425089446)

what about unrelated stuff

In [44]:
evaluate_cobrinha('java','arrays',tag_vectors_index,sorted_similarity_dict, global_similarity_index) 

(0.11558132868932458, 0.62033547900763397)

In [45]:
evaluate_cobrinha('ruby','python-2.7',tag_vectors_index,sorted_similarity_dict, global_similarity_index) 

(-0.012370419948397549, 0.64212728227082461)

In [46]:
evaluate_cobrinha('database','python-2.7',tag_vectors_index,sorted_similarity_dict, global_similarity_index) 

(0.020250833718711614, 0.64969034610153675)

what's the mean and stddev of the similarity between all tags and each other?

In [47]:
running_avgs = []

for (tag, similarities_to_other_tags) in similarity_dict.items():
    sims = [tpl[1] for tpl in similarities_to_other_tags]
    
    avg = np.array(sims).mean()
    
    running_avgs.append(avg)
    
np.array(running_avgs).mean(),np.array(running_avgs).std()

(0.31303853122620334, 0.064948750968199978)